In [ ]:
import json
import numpy as np
import os
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from app.learner import Learner
from app import settings


In [ ]:
dataDir = "../data/universal-trending-20171129-1548/"
learner = Learner(dataDir)
learner.vectors.shape

In [ ]:
print("Features with highest doc frequency")
sorted(zip(learner.vectorizer.idf_, learner.vectorizer.vocabulary_), key=lambda x: x[0])[:5]

In [ ]:
print("Features with lowest doc frequency")
sorted(zip(learner.vectorizer.idf_, learner.vectorizer.vocabulary_), key=lambda x: -x[0])[:5]

In [ ]:
print(settings.DITHERING_EPSILON)
learner.dither(list(range(10)))

In [ ]:
from __future__ import print_function
from ipywidgets import interact, fixed

settings.BATCH_SIZE = 1
settings.EXPLORE_PROBABILITY = 0.0
settings.DITHERING_EPSILON = 20

def handleResponse(x, docId):
    if x != 0.5:
        learner.handleResponse(docId, x)
        getFeedback()

learner.candidates = []
        
def getFeedback():
    if not learner.candidates:
        learner.candidates = learner.getCandidates()
    candidate = learner.candidates.pop(0)
    print("\n", candidate["title"])
    print(candidate["description"])
    interact(handleResponse, x=(0.0,1,0.5), docId=fixed(candidate["id"]));
    
getFeedback()